<a href="https://colab.research.google.com/github/AnIsAsPe/LDATopicModeling_pyspark/blob/main/Introducci%C3%B3n_a_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



![image.png](https://spark.apache.org/docs/1.1.1/img/cluster-overview.png)

# Instalación de PySpark en Colab

In [1]:
!pip install pyspark

Más información sobre como instalar PySpark en ambientes locales se puede consultar [aquí](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

# Spark Context

In [53]:
# Crear SparkContext para conectar con el cluster
from pyspark import SparkConf
from pyspark import SparkContext
import numpy as np

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[4]"))

# "the master" es la computadora conectada con el resto de las computadoras en el cluster que
#  administra la división y transformación de los datos

# Resilent Distributed Datasets

## Crear un objeto RDD paralelizando una colección

In [2]:
!cat /proc/cpuinfo | grep processor | wc -l #procesadores ebn máquina colab


2


In [3]:
!echo $(($(getconf _PHYS_PAGES) * $(getconf PAGE_SIZE) / (1024 * 1024))) #RAM


12991


In [4]:
a_np = np.random.randint(0,100,20)
a_rdd = sc.parallelize(a_np, 4)

In [5]:
print(type(a_np))
print(type(a_rdd))

<class 'numpy.ndarray'>
<class 'pyspark.rdd.RDD'>


In [6]:
a_rdd.collect()  # regresa los elementos distribuidos 

[76, 75, 11, 5, 55, 50, 11, 81, 39, 5, 29, 23, 12, 90, 54, 26, 45, 47, 93, 3]

In [7]:
a_rdd.glom().collect()  # con glom podemos ver como se hicieron las particiones

[[76, 75, 11, 5, 55],
 [50, 11, 81, 39, 5],
 [29, 23, 12, 90, 54],
 [26, 45, 47, 93, 3]]

## Crear un objeto RDD a partir de datos externos

El texto esta disponible [aquí](https://github.com/AnIsAsPe/Estadistica_y_Probabilidad_para-CD-/blob/main/Semana1/gabriel_garcia_marquez_cien_annos_soledad.txt)

In [52]:
file = '/content/drive/MyDrive/Datos/gabriel_garcia_marquez_cien_annos_soledad.txt'
texto = sc.textFile(file, 4)


In [10]:
type(texto)

pyspark.rdd.RDD

In [9]:
texto.map(lambda s: len(s)).reduce(lambda a, b: a + b) # cantidad de caracteres

814791

# Operaciones en RDD
- Transformaciones (Map)
- Acciones (Reduce)

### Transformaciones (Map)

In [11]:
#map function
sc.parallelize([3,4,5]).map(lambda x: range(1,x)).collect()

[range(1, 3), range(1, 4), range(1, 5)]

In [12]:
#flatmap regresa el resultado en una sola lista
sc.parallelize([3,4,5]).flatMap(lambda x: [x, x*x]).collect()

[3, 9, 4, 16, 5, 25]

In [22]:
#mapping con funciones regulares
def square_if_odd(x):
    """
    Si el numero es non, regresa el cuadrado, los pares en cambio
    no sufren transformación
    """
    if x % 2==1:
        return x*x
    else:
        return x

numeros = sc.parallelize(np.arange(20))
numeros.map(square_if_odd).collect()

[0, 1, 2, 9, 4, 25, 6, 49, 8, 81, 10, 121, 12, 169, 14, 225, 16, 289, 18, 361]

###  Acciones (Reduce)

In [30]:
numeros2 = sc.parallelize([1, 4, 6, 2, 9, 10])
sum = numeros2.reduce(lambda a, b : a + b)
sum

32

In [33]:
numeros2.count() #contar los elementos

6

In [34]:
numeros2.first()

1

In [35]:
numeros2.take(3)

[1, 4, 6]

In [37]:
#Encontrar el elemento máximo con reduce
numeros2.reduce(lambda x,y: x if x > y else y)

10

In [38]:
#Filtros: ejemplo devuelve numeros positivos divisibles entre 3
numeros2.filter(lambda x:x%3==0 and x>=0).collect()

[6, 9]

In [39]:

palabras = 'MapReduce, GFS, Hadoop, HDFS, Spark'.split(',')

palabrasRDD = sc.parallelize(palabras)

palabrasRDD.reduce(lambda w,v: w if len(w) >len(v) else v)

'MapReduce'

In [21]:
#uso de funciones comunes utilizando reduce 
def largerThan(x,y):
    """
    Regresa la palabra más larga
    """
    if len(x)> len(y):
        return x
    else:
        return y

wordRDD.reduce(largerThan)

'MapReduce'

# Groupby

In [43]:
numeros.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [51]:
result=numeros.groupBy(lambda x:x%2).collect()
result

[(0, <pyspark.resultiterable.ResultIterable at 0x7f06f7bd1c90>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f06f7b68fd0>)]

In [45]:
sorted([(x, sorted(y)) for (x, y) in result])

[(0, [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]),
 (1, [1, 3, 5, 7, 9, 11, 13, 15, 17, 19])]

## Lazy Evaluation 
Es una estrategia de spark para acelerar operaciones paralelizadas.
Deja lista una secuencia de tareas paso por paso en una tarea pero retrasa la ejecución hasta que es absolutamente necesaria.

(ejemplo en https://dzone.com/articles/the-benefits-amp-examples-of-using-apache-spark-wi)





Documentación oficial de spark: 

* [RDD Programming Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#initializing-spark)

* [Cluster Mode Oberview](https://databricks.com/blog/2018/05/03/benchmarking-apache-spark-on-a-single-node-machine.html)





